# Обучение модели для сегментации мозга

## Подготовка среды

In [ ]:
import datetime
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation


plt.rcParams["animation.html"] = "jshtml"
import tensorflow as tf
from loader_data import PreprocessLoadData, AnimateView
from models.model_MultiReaUNet3d import Model3DMultiResUnet
from models.model_UNet3d import Model3DUnet
import numpy as np

gpu_device = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu_device, True)



## Определяем ключевые параметры

 - **KERNEL_DIM** - Размер изображения которое подаем на вход сети. Независимо от размера исходного изображения, генератор будет делить исходное изображение на куски размером KERNEK_DIM
 - **STEP_WINDOW** - Разбиение исходного изображения на куски размером KERNEK_DIM происходит по принципу плавающего окна. STEP_WINDOW задает шаг смещения окна.
 - **BATCH_SIZE** - Количество кусков в одном батче
 - **Model** - Класс модели, которую будем обучать.

In [ ]:
STEP_WINDOW = (24, 24, 24)
BATCH_SIZE = 5
KERNEL_DIM = (64, 64, 64)

Model = Model3DUnet


## Проверяем наличие данных для обучения

### Находим обучающие данные 

In [ ]:

loader = PreprocessLoadData(kernel_size=KERNEL_DIM, step=STEP_WINDOW, batch_size=BATCH_SIZE)
loader.find_files()
print(f"Был найден набор из {loader.length_data()} пар данных")


### Получаем генератор данных

In [ ]:
train_dataset = loader.get_generator_data("train",
                                          threshold=0.05,
                                          random_change_plane=False,
                                          augmentation=False,
                                          is_whitening=False
                                         )
test_dataset = loader.get_generator_data("test",
                                         threshold=0.05,
                                         random_change_plane=False,
                                         augmentation=False,
                                         is_whitening=False)


#### Проверяем как работает генератор

In [ ]:
def display(display_list):
    fig = plt.figure(figsize=(15, 5))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    list_ax = [fig.add_subplot(1, len(display_list), i + 1) for i in range(len(display_list))]
    list_exponent = []
    for i in range(len(display_list)):
        plt.title(title[i])
        list_exponent.append((display_list[i].min(), display_list[i].max()))
    list_ims = []
    for ind_z in range(display_list[0].shape[0]):
        if display_list[0][ind_z,...].max() == 0:
            continue
        ims = []
        for i in range(len(display_list)):
            cmap_val = 'gray' if i == 0 else "viridis"
            im = list_ax[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i][ind_z,...]),
                           animated=True, cmap=cmap_val, vmin=list_exponent[i][0], vmax=list_exponent[i][0])
            ims.append(im)
        list_ims.append(ims)
    ani = animation.ArtistAnimation(fig,
                                    list_ims,
                                    interval=50,
                                    blit=True)
    return ani


for image, mask in train_dataset:
    print(image.shape)
    print(mask.shape)
    print(image.max())
    print(mask.max())
    sample_image, sample_mask = image, mask
    break
    
    
display([sample_image[0], sample_mask[0]])

## Создаем модель

 - n_channels: Размер канала входных данных.
 - initial_learning_rate: Начальная скорость обучения модели.
 - n_classes: Количество классов, которые изучает модель .
 - start_val_filters: Количество фильтров, которые будет иметь первый слой в сети.
 - list_metrics: Список метрик, для обучения модели.
 - type_up_convolution: тип повышающего слоя. up_sampling_3d использует меньше памяти.
 - pool_size: Размер пула для максимальных операций объединения. Целое число или кортеж.
 - input_img_shape: Форма входных данных кортеж(3N) или целое число (если форма соответствует кубу).
 - depth: глубина модели. Уменьшение глубины может уменьшить объем памяти, необходимый для тренировки.


In [ ]:
manager_model = Model(depth=5, start_val_filters=16, input_img_shape=KERNEL_DIM, n_classes=3, n_channels=3,
                                    initial_learning_rate=1.0)


### Обучаем

In [ ]:

EPOCHS = 150

total_pieces = (192 - KERNEL_DIM[-3]) // STEP_WINDOW[-3] + 1
total_pieces *= (256 - KERNEL_DIM[-2]) // STEP_WINDOW[-2] + 1
total_pieces *= (256 - KERNEL_DIM[-1]) // STEP_WINDOW[-1] + 1

total_mri_study = loader.length_data() * 0.8
total_mri_study = 2

STEPS_PER_EPOCH = round((total_pieces * total_mri_study) / BATCH_SIZE)


VALIDATION_STEPS = round(2)

checkpoint_path = 'checkpoints3D//cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_directory = os.path.join("Results", timestamp)

# Добавление отображения через TensorBoard.
log_dir = os.path.join("Results",
                       timestamp,
                       "logs",
                       "fit",
                       datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
full_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       save_weights_only=True,
                                       save_freq=STEPS_PER_EPOCH,
                                       verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                   histogram_freq=1,
                                   profile_batch=0,
                                   write_images=True),
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=25)]

model_history = manager_model.model.fit(train_dataset, epochs=EPOCHS,
                                        steps_per_epoch=STEPS_PER_EPOCH,
                                        validation_steps=VALIDATION_STEPS,
                                        validation_data=test_dataset,
                                        verbose=1,
                                        callbacks=full_callbacks)

manager_model.model.save(f"train.{Model.__name__}.{timestamp}")


In [ ]:
%%capture
for image, mask in test_dataset:
    pred_mask = manager_model.model.predict(image)
    # используйте порог, чтобы получить точные прогнозы
    threshold = 0.01
    pred_mask[pred_mask > threshold] = 1.0
    pred_mask[pred_mask <= threshold] = 0.0
    print(f"pred_mask.shape={pred_mask.shape}")
    if len(pred_mask.shape) == 4:
        # Область усиления контраста
        gd_enhancing_tumor_ani = AnimateView().get_ani(pred_mask[:, :, :, 2])
        # отек
        peritumoral_edema_ani = AnimateView().get_ani(pred_mask[:, :, :, 1])
        # ядро некротической и неусиливающейся опухоли
        ncr_net_ani = AnimateView().get_ani(pred_mask[:, :, :, 0])
        # T1_MRI
        t1_mri_ani = AnimateView().get_ani(image[:, :, :, 0])
        # T1_ce_MRI
        t1ce_mri_ani = AnimateView().get_ani(image[:, :, :, 1])
        # T2_MRI
        t2_mri_ani = AnimateView().get_ani(image[:, :, :, 2])
        # Оригинальная маска_0
        orign_mask_ani_0 = AnimateView().get_ani(mask[:, :, :, 0])
        # Оригинальная маска_1
        orign_mask_ani_1 = AnimateView().get_ani(mask[:, :, :, 1])
        # Оригинальная маска_2
        orign_mask_ani_2 = AnimateView().get_ani(mask[:, :, :, 2])
    elif len(pred_mask.shape) == 5:
        # Область усиления контраста
        gd_enhancing_tumor_ani = AnimateView().get_ani(pred_mask[0, :, :, :, 2])
        # peritumoral edema
        peritumoral_edema_ani = AnimateView().get_ani(pred_mask[0, :, :, :, 1])
        # NCR/NET
        ncr_net_ani = AnimateView().get_ani(pred_mask[0, :, :, :, 0])
        # T1_MRI
        t1_mri_ani = AnimateView().get_ani(image[0, :, :, :, 0])
        # T1_ce_MRI
        t1ce_mri_ani = AnimateView().get_ani(image[0, :, :, :, 1])
        # T2_MRI
        t2_mri_ani = AnimateView().get_ani(image[0, :, :, :, 2])
        # Оригинальная маска_0
        orign_mask_ani_0 = AnimateView().get_ani(mask[0, :, :, :, 0])
        # Оригинальная маска_1
        orign_mask_ani_1 = AnimateView().get_ani(mask[0, :, :, :, 1])
        # Оригинальная маска_2
        orign_mask_ani_2 = AnimateView().get_ani(mask[0, :, :, :, 2])
    break


#### T1 МРТ

In [ ]:
t1_mri_ani

#### T1 Контраст МРТ

In [ ]:
%%capture
t1ce_mri_ani

#### T2 МРТ

In [ ]:
%%capture
t2_mri_ani

## Результат предсказания


### Область отека


In [ ]:
peritumoral_edema_ani

### Область усиления контраста

In [ ]:
gd_enhancing_tumor_ani

### ядро некротической и неусиливающейся опухоли

In [ ]:
ncr_net_ani

### Оригинальная маска - Как должно быть

In [ ]:
orign_mask_ani_1

In [ ]:
orign_mask_ani_2

In [ ]:
orign_mask_ani_0